link to original notebook
*note: this notebook is missing a signifcant amount of code an will not run on it's own*
https://colab.research.google.com/github/idiap/w2v2-air-traffic/blob/main/src/eval_xlsr_atc_model.ipynb#scrollTo=7ddc6ef1-5a73-40c8-8199-deb3e94bdb2b

In [14]:
import os

from bs4 import BeautifulSoup

import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import AutoModelForCTC, Wav2Vec2Processor, AutoProcessor, pipeline
import torchaudio.functional as F

from evaluate import load

if os.name == 'posix':
    # Linux
    os.chdir('/home/rodoggx/Documents/iResearch-2024/DATA')
elif os.name == 'nt':
    # Windows
    os.chdir('C:/Users/rowan/OneDrive/Desktop/code/iResearch-2024/DATA')
else:
    print("Unknown operating system")

In [11]:
audioFiles = [f for f in os.listdir() if f.endswith('.wav')]
xmlFiles = []
for i in range(len(audioFiles)):
	xmlFiles.append(audioFiles[i].replace('.wav', '.xml'))

transcripts = []

datasetDict = {
	'audio': audioFiles,
	'transcripts': []
}

for i in range(len(xmlFiles)):
	with open(xmlFiles[i], 'r', encoding='utf-8') as file:
		data = file.read()

	Bs_data = BeautifulSoup(data, "xml")
	b_unique = Bs_data.find_all('text')
	for j in b_unique:
		transcripts.append(j.text)
	datasetDict['transcripts'].append(transcripts)
	transcripts = []

df = pd.DataFrame(datasetDict)

df


,audio,transcripts
0,LKPR_RUZYNE_Radar_120_520MHz_20201026_205056.wav,[KLM Seven Three Whiskey maintain present head...
1,LKPR_RUZYNE_Tower_134_560MHz_20201025_101021.wav,[Ryan Air Seven Romeo Mike route reaching and ...
2,LZIB_STEFANIK_Tower_118_3MHz_20210505_145115.wav,[Topjet Four Zero Two via delta foxtrot ten th...
3,LKTB_BRNO_Approach-Radar_127_350MHz_20201028_1...,[Oscar Kilo Delta Sierra India turn left headi...
4,LKPR_RUZYNE_Tower_134_560MHz_20201025_204809.wav,[Ruzyne tower Trans Europe Four Two One [Ne Cz...
...,...,...
555,YSSY_SYDNEY_Tower_120_5MHz_20210502_055020.wav,[[#unnamed]tower[/#unnamed] [#unnamed]good[/#u...
556,LSZH_ZURICH_Tower_118_1MHz_20210414_101150.wav,[Hotel Delta Lima contact apron one two one de...
557,YSSY_SYDNEY_Tower_120_5MHz_20210505_124859.wav,[[#command]cleared[/#command] [#command]to[/#c...
558,YSSY_SYDNEY_Tower_120_5MHz_20210501_090436.wav,[Sydney tower evening Qantas Seventy Seven Zer...


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\rowan\OneDrive\Desktop\code\iResearch-2024\venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


In [17]:
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="Jzuluaga/wav2vec2-xls-r-300m-en-atc-uwb-atcc-and-atcosim")

result = pipe(df['audio'][0])
print(result['text'])
print(df['transcripts'][0])

wer = load('wer')
wer_score = wer.compute(predictions=[result['text']], references=[df['transcripts'][0]])
print(wer_score)
# print(result['text'] == df['transcripts'][0])

Some weights of the model checkpoint at Jzuluaga/wav2vec2-xls-r-300m-en-atc-uwb-atcc-and-atcosim were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Jzuluaga/wav2vec2-xls-r-300m-en-atc-uwb-atcc-and-atcosim and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrization

lm seven three whiskey maintain present heading for vectoring ils approach runway two four and rehe our speed no below two six zero knots
['KLM Seven Three Whiskey maintain present heading for vectoring for ILS approach runway two four and reduce your speed now below two six zero knots']


ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: ['lm seven three whiskey maintain present heading for vectoring ils approach runway two four and rehe our speed no below two six zero knots'],
Input references: [['KLM Seven Three Whiskey maintain present heading for vectoring for ILS approach runway two four and reduce your speed now below two six zero knots']]